In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
from functools import reduce

In [2]:
def bold_significance(val):
    if type(val)==str:
        val = float(val.replace('<', ''))
    fontweight = 'bold' if val<0.05 else 'normal'
    return 'font-weight: {}'.format(fontweight)

In [3]:
# Load in data
bb_data = pd.read_csv('bbdata_area.csv')
bb_data.set_index('pat_id',inplace=True)
bb_data = bb_data[bb_data.index!=15]

# Calculate effective diameter from area
bb_data['effective_diameter'] = bb_data['area'].apply(lambda x: np.sqrt((4 * x)/np.pi))

# Calculate percent oversizing
bb_data['percent_oversizing'] = 100*((bb_data['graft'] / bb_data['effective_diameter'])-1)

# Curvature * diameter metric
bb_data['CD'] = bb_data['effective_diameter'] * bb_data['curve']

# Group by BBH
bb_data['group'] = bb_data['bbh'].apply(lambda x: 'BB' if x >= 5 else 'NBB')

results_data_all = bb_data.copy()
results_data_all['group'] = 'All'
results_data = pd.concat([bb_data, results_data_all])


results_data.rename(columns={
                                'bbh': 'BBH (mm)', 
                                'curve':'Curvature (mm-1)', 
                                'effective_diameter':'Diameter (mm)', 
                                'percent_oversizing':'Graft Oversizing (%)',
                                'bba': 'BBA (deg)',
                                'bbl': 'BBL (mm)',
                                'graft': 'Proximal Graft Diameter (mm)',
                                'area': 'Aortic Area (mm2)'
    
                             }, inplace=True)
results_data_output = results_data.groupby('group').agg([np.mean, np.std]).reset_index()
results_data_output



group  BBH (mm)            BBL (mm)            BBA (deg)            \
             mean       std      mean       std       mean       std   
0   All  4.714757  2.753160  5.013980  4.528136  23.700877  9.811713   
1    BB  6.975649  1.655657  8.755136  3.160407  27.716454  8.209978   
2   NBB  2.453864  1.378984  1.272825  1.481658  19.685300  9.999964   

  Proximal Graft Diameter (mm)           Aortic Area (mm2)              \
                          mean       std              mean         std   
0                        36.85  4.976524        918.097052  207.724023   
1                        38.10  3.573047       1002.374002  118.099846   
2                        35.60  6.003703        833.820103  247.713069   

  Curvature (mm-1)           Diameter (mm)           Graft Oversizing (%)  \
              mean       std          mean       std                 mean   
0         0.035277  0.013405     33.955647  4.100261             8.695370   
1         0.039816  0.014071     35.668192  2.119310             6.965946   
2         0.030738  0.011644     32.243102  4.948077            10.424794   

                   CD            
        std      mean       std  
0  8.081185  1.191797  0.478355  
1  9.617021  1.418725  0.511669  
2  6.223641  0.964870  0.326849

In [28]:
bb_data

,bbh,bbl,bba,graft,area,curve,effective_diameter,percent_oversizing,CD,group
pat_id,,,,,,,,,,
1,8.634628,8.250183,43.390292,40,898.227750,0.052182,33.818029,18.280104,1.764706,BB
2,3.283748,0.588213,36.603597,34,773.395274,0.056010,31.380208,8.348549,1.757614,NBB
3,1.634992,0.875245,24.452053,45,1140.870716,0.025875,38.113012,18.069912,0.986172,NBB
5,6.627722,9.727552,21.997878,34,1056.207751,0.056139,36.671589,-7.285175,2.058692,BB
6,5.709863,5.964424,27.693259,34,816.486608,0.036920,32.242566,5.450666,1.190407,BB
7,1.431439,0.255345,25.505892,28,464.487064,0.042128,24.318785,15.137332,1.024496,NBB
9,9.303285,9.591820,41.378818,34,1044.351776,0.067791,36.465189,-6.760389,2.472014,BB
10,0.831111,0.591159,9.066720,31,599.295628,0.036416,27.623303,12.224087,1.005926,NBB
11,8.889974,12.260861,28.735107,37,1003.028189,0.035130,35.736468,3.535693,1.255420,BB


In [29]:
results_data_output
one_sigfig = ['BBL (mm)', 'BBH (mm)','Diameter (mm)','CD']
three_sigfig = ['Curvature (mm-1)']
results_temp = []
cols = ['Proximal Graft Diameter (mm)','Aortic Area (mm2)','Curvature (mm-1)', 'Diameter (mm)', 'CD', 'Graft Oversizing (%)','BBL (mm)', 'BBH (mm)', 'BBA (deg)']
for col in cols:
    fstring = "{0:.3f} ± {1:.3f}" if col in three_sigfig else ("{0:.1f} ± {1:.1f}" if col in one_sigfig else "{0:.0f} ± {1:.0f}")
    mean_std = results_data_output[col]
    mean_std['mean ± std'] = mean_std.apply(lambda x: fstring.format(x['mean'], x['std']),axis=1)
    mean_std.drop(columns=['mean', 'std'],inplace=True)
    mean_std.index = ['All', 'BB', 'NBB']
    mean_std.columns = pd.MultiIndex.from_product([[col], mean_std.columns])
    results_temp.append(mean_std)
results_data_output = reduce(lambda left, right: pd.merge(left, right, left_index=True, right_index=True),results_temp)
display(results_data_output)    
for feat in cols:
    bbgroup = results_data[results_data['group'] == 'BB']
    nobbgroup = results_data[results_data['group'] == 'NBB']
    s, p = stats.ttest_ind(bbgroup[feat], nobbgroup[feat])
    results_data_output.loc['p-value', feat] = ['{:.3f}'.format(p)]
results_data_output.fillna('',inplace=True)
    
results_data_output = results_data_output.transpose()
results_data_output = results_data_output.reindex(sorted(results_data_output.index.values))
results_data_output.style.applymap(bold_significance, subset='p-value')
#writer = pd.ExcelWriter("/Users/maxfrohlich/Dropbox/Stanford-SJSU-Manuscript/Manuscript/Figures/bb_groups_demo.xlsx")
#results_data_output.to_excel(writer, 'sheet1')


/Users/maxfrohlich/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/maxfrohlich/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


,Proximal Graft Diameter (mm),Aortic Area (mm2),Curvature (mm-1),Diameter (mm),CD,Graft Oversizing (%),BBL (mm),BBH (mm),BBA (deg)
,mean ± std,mean ± std,mean ± std,mean ± std,mean ± std,mean ± std,mean ± std,mean ± std,mean ± std
All,37 ± 5,918 ± 208,0.035 ± 0.013,34.0 ± 4.1,1.2 ± 0.5,9 ± 8,5.0 ± 4.5,4.7 ± 2.8,24 ± 10
BB,38 ± 4,1002 ± 118,0.040 ± 0.014,35.7 ± 2.1,1.4 ± 0.5,7 ± 10,8.8 ± 3.2,7.0 ± 1.7,28 ± 8
NBB,36 ± 6,834 ± 248,0.031 ± 0.012,32.2 ± 4.9,1.0 ± 0.3,10 ± 6,1.3 ± 1.5,2.5 ± 1.4,20 ± 10


,,All,BB,NBB,p-value
Aortic Area (mm2),mean ± std,918 ± 208,1002 ± 118,834 ± 248,0.068
BBA (deg),mean ± std,24 ± 10,28 ± 8,20 ± 10,0.065
BBH (mm),mean ± std,4.7 ± 2.8,7.0 ± 1.7,2.5 ± 1.4,0.000
BBL (mm),mean ± std,5.0 ± 4.5,8.8 ± 3.2,1.3 ± 1.5,0.000
CD,mean ± std,1.2 ± 0.5,1.4 ± 0.5,1.0 ± 0.3,0.030
Curvature (mm-1),mean ± std,0.035 ± 0.013,0.040 ± 0.014,0.031 ± 0.012,0.133
Diameter (mm),mean ± std,34.0 ± 4.1,35.7 ± 2.1,32.2 ± 4.9,0.059
Graft Oversizing (%),mean ± std,9 ± 8,7 ± 10,10 ± 6,0.352
Proximal Graft Diameter (mm),mean ± std,37 ± 5,38 ± 4,36 ± 6,0.273


In [4]:
bb_data
stats.pearsonr(bb_data['bbh'], bb_data['bbl'])

(0.9192792446264377, 3.943269304222449e-09)

In [7]:
results_data_corr = results_data[results_data.group == 'All']
bba_out = []
bbl_out = []
bbh_out = []
for feat in ['Proximal Graft Diameter (mm)', 'Aortic Area (mm2)', 'Curvature (mm-1)', 'Diameter (mm)', 'Graft Oversizing (%)', 'CD']:
    current_feat = results_data_corr[feat]
    bba = results_data_corr['BBA (deg)']
    bbl = results_data_corr['BBL (mm)']
    bbh = results_data_corr['BBH (mm)']
    bba_r, bba_p = stats.pearsonr(current_feat, bba)
    bbl_r, bbl_p = stats.pearsonr(current_feat, bbl)
    bbh_r, bbh_p = stats.pearsonr(current_feat, bbh)
    bba_out.append(pd.DataFrame({'r-value': [bba_r], 'p-value':[bba_p]},index=[feat]))
    bbl_out.append(pd.DataFrame({'r-value': [bbl_r], 'p-value':[bbl_p]},index=[feat]))
    bbh_out.append(pd.DataFrame({'r-value': [bbh_r], 'p-value':[bbh_p]},index=[feat]))
bbl_corr = pd.concat(bbl_out)
bba_corr = pd.concat(bba_out)
bbh_corr = pd.concat(bbh_out)
bbal_corr = pd.concat({'BBL Correlation':bbl_corr.sort_index(), 
                       'BBA Correlation': bba_corr.sort_index(),
                       'BBH Correlation': bbh_corr.sort_index()})

#bbal_corr.to_csv('/Users/maxfrohlich/Dropbox/figure_1_raw/bba_bbl.csv')


bbal_corr.style.applymap(bold_significance, subset='p-value')